In [22]:
# !pip install tensorflow-gpu


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking, Reshape, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [24]:
import pandas as pd

test_file_path = r'C:\Users\Jean\Documents\EIGSI\4a\Python for datascience\PROJET STARCRAFT\TEST.CSV'
train_file_path = r'C:\Users\Jean\Documents\EIGSI\4a\Python for datascience\PROJET STARCRAFT\TRAIN.CSV'

test_df = pd.read_csv(test_file_path, sep=',', on_bad_lines='skip')
train_df = pd.read_csv(train_file_path, sep=',', on_bad_lines='skip')
print("\nTrain DataFrame:")
print(train_df.head())

print("Test DataFrame:")
print(test_df.head())




C:\Users\Jean\AppData\Local\Temp\ipykernel_7352\2595427852.py:6: DtypeWarning: Columns (3376,3377,3378,3379,3380,3381,3382,3383,3384,3385,3386,3387,3388,3389,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399,3400,3401,3402,3403,3404,3405,3406,3407,3408,3409,3410,3411,3412,3413,3414,3415,3416,3417,3418,3419,3420,3421,3422,3423,3424,3425,3426,3427,3428,3429,3430,3431,3432,3433,3434,3435,3436,3437,3438,3439,3440,3441,3442) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(test_file_path, sep=',', on_bad_lines='skip')



Train DataFrame:
  http://eu.battle.net/sc2/en/profile/4234852/1/First/  Protoss  Base     s  \
0  http://eu.battle.net/sc2/en/profile/3074362/1/...    Protoss     s  Base   
1  http://eu.battle.net/sc2/en/profile/4234852/1/...    Protoss  Base     s   
2  http://eu.battle.net/sc2/en/profile/3074362/1/...    Protoss  Base     s   
3  http://eu.battle.net/sc2/en/profile/4234852/1/...    Protoss  Base     s   
4  http://eu.battle.net/sc2/en/profile/3074362/1/...    Protoss     s  Base   

  s.1   s.2   s.3 s.4        t5    Base.1  ... t1650 s.671 hotkey10.62 s.672  \
0   s     s  Base   s         s      Base  ...   NaN   NaN         NaN   NaN   
1   s     s  Base   s  hotkey30  hotkey00  ...   NaN   NaN         NaN   NaN   
2   s  Base     s   s         s        t5  ...   NaN   NaN         NaN   NaN   
3   s     s  Base   s  hotkey30  hotkey00  ...   NaN   NaN         NaN   NaN   
4   s     s  Base   s  hotkey30         s  ...   NaN   NaN         NaN   NaN   

  s.673 hotkey32.400 hotke

C:\Users\Jean\AppData\Local\Temp\ipykernel_7352\2595427852.py:7: DtypeWarning: Columns (2559,2560,2561,2562,2563,2564) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_file_path, sep=',', on_bad_lines='skip')


In [48]:
print (len(test_df))

264


In [25]:
num_columns_test = test_df.shape[1]
num_columns_train = train_df.shape[1]


#on ajoute une ligne pour nommer les colonnes

new_columns_test = ['nom'] + [f'action {i}' for i in range(1, num_columns_test)]
new_columns_train = ['nom'] + [f'action {i}' for i in range(1, num_columns_train)]

# Assigner les nouveaux noms de colonnes aux DataFrames
test_df.columns = new_columns_test
train_df.columns = new_columns_train



print("\nTrain DataFrame:")
print(train_df.head())


Train DataFrame:
                                                 nom action 1 action 2  \
0  http://eu.battle.net/sc2/en/profile/3074362/1/...  Protoss        s   
1  http://eu.battle.net/sc2/en/profile/4234852/1/...  Protoss     Base   
2  http://eu.battle.net/sc2/en/profile/3074362/1/...  Protoss     Base   
3  http://eu.battle.net/sc2/en/profile/4234852/1/...  Protoss     Base   
4  http://eu.battle.net/sc2/en/profile/3074362/1/...  Protoss        s   

  action 3 action 4 action 5 action 6 action 7  action 8  action 9  ...  \
0     Base        s        s     Base        s         s      Base  ...   
1        s        s        s     Base        s  hotkey30  hotkey00  ...   
2        s        s     Base        s        s         s        t5  ...   
3        s        s        s     Base        s  hotkey30  hotkey00  ...   
4     Base        s        s     Base        s  hotkey30         s  ...   

  action 2555 action 2556 action 2557 action 2558 action 2559 action 2560  \
0        

In [26]:
#Ici on isole le nom des joueurs de l'url de leur compte

train_df[train_df.columns[0]] = train_df[train_df.columns[0]].astype(str)
train_df[train_df.columns[0]] = train_df[train_df.columns[0]].str.split('/').str[-2]
print(train_df.head())
print(test_df.head())

        nom action 1 action 2 action 3 action 4 action 5 action 6 action 7  \
0  Stardust  Protoss        s     Base        s        s     Base        s   
1     First  Protoss     Base        s        s        s     Base        s   
2  Stardust  Protoss     Base        s        s     Base        s        s   
3     First  Protoss     Base        s        s        s     Base        s   
4  Stardust  Protoss        s     Base        s        s     Base        s   

   action 8  action 9  ... action 2555 action 2556 action 2557 action 2558  \
0         s      Base  ...         NaN         NaN         NaN         NaN   
1  hotkey30  hotkey00  ...         NaN         NaN         NaN         NaN   
2         s        t5  ...         NaN         NaN         NaN         NaN   
3  hotkey30  hotkey00  ...         NaN         NaN         NaN         NaN   
4  hotkey30         s  ...         NaN         NaN         NaN         NaN   

  action 2559 action 2560 action 2561 action 2562 action 2563 

In [27]:
#Ici on remplit les NaN par unkown (pour par la suite pouvoir faire un masquage)
train_df = train_df.fillna('unknown')
test_df = test_df.fillna('unknown')
print(train_df.head())
print(test_df.head())

        nom action 1 action 2 action 3 action 4 action 5 action 6 action 7  \
0  Stardust  Protoss        s     Base        s        s     Base        s   
1     First  Protoss     Base        s        s        s     Base        s   
2  Stardust  Protoss     Base        s        s     Base        s        s   
3     First  Protoss     Base        s        s        s     Base        s   
4  Stardust  Protoss        s     Base        s        s     Base        s   

   action 8  action 9  ... action 2555 action 2556 action 2557 action 2558  \
0         s      Base  ...     unknown     unknown     unknown     unknown   
1  hotkey30  hotkey00  ...     unknown     unknown     unknown     unknown   
2         s        t5  ...     unknown     unknown     unknown     unknown   
3  hotkey30  hotkey00  ...     unknown     unknown     unknown     unknown   
4  hotkey30         s  ...     unknown     unknown     unknown     unknown   

  action 2559 action 2560 action 2561 action 2562 action 2563 

In [28]:
# Convertir toutes les colonnes de touches en chaînes de caractères
train_df = train_df.astype(str)
test_df = test_df.astype(str)


In [29]:
# Initialiser le label encoder pour les touches pour que le model puisse utiliser ces valeurs pour s'entrainer
touch_encoder = LabelEncoder()

# Concatenater les données d'entraînement et de test pour s'assurer que l'encodage est cohérent
all_touches = pd.concat([train_df.drop(columns=['nom']), test_df.drop(columns=['nom'])], axis=0).stack().unique()
touch_encoder.fit(all_touches)

# Encoder les touches dans les DataFrames
for column in train_df.columns[1:]:
    train_df[column] = touch_encoder.transform(train_df[column])
for column in test_df.columns[1:]:
    test_df[column] = touch_encoder.transform(test_df[column])

# Initialiser le label encoder pour les noms
name_encoder = LabelEncoder()

# Concatenater les noms des jeux d'entraînement et de test pour s'assurer que l'encodage est cohérent
all_names = pd.concat([train_df['nom'], test_df['nom']], axis=0).unique()
name_encoder.fit(all_names)

# Encoder les noms en valeurs numériques dans les jeux de données d'entraînement et de test
y_train = name_encoder.transform(train_df['nom'])
y_test = name_encoder.transform(test_df['nom'])


In [30]:
# Vérifier que toutes les classes sont bien encodées
num_classes = len(name_encoder.classes_)
assert len(np.unique(y_train)) <= num_classes
assert len(np.unique(y_test)) <= num_classes


In [31]:
#supprime la colonne nom des X car c'est y que l'on veut predire (la colonne des noms)

X_test = test_df.drop(columns=['nom']).values
X_train = train_df.drop(columns=['nom']).values

In [32]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Définir la longueur maximale des séquences
max_length = 2564  

# Remplir les séquences pour qu'elles aient la même longueur
# Ici, nous utilisons la valeur encodée de 'unknown' comme valeur de remplissage pour le masque 
unknown_value = touch_encoder.transform(['unknown'])[0]
X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post', value=unknown_value)
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post', value=unknown_value)

# Créer des masques
train_masks = np.array([[1 if token != unknown_value else 0 for token in seq] for seq in X_train_padded])
test_masks = np.array([[1 if token != unknown_value else 0 for token in seq] for seq in X_test_padded])

# Vérifier les formes des données
print(X_train_padded.shape, train_masks.shape)
print(X_test_padded.shape, test_masks.shape)

(2236, 2564) (2236, 2564)
(264, 2564) (264, 2564)


In [33]:
# Vérifier que toutes les classes sont bien encodées
num_classes = len(name_encoder.classes_)
assert len(np.unique(y_train)) <= num_classes
assert len(np.unique(y_test)) <= num_classes

# Convertir les étiquettes en one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)


In [34]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking, Reshape, Dropout, BatchNormalization

# Définir les entrées du modèle
input_seq = Input(shape=(max_length,))
# Ajouter une dimension supplémentaire
reshaped_input = Reshape((max_length, 1))(input_seq)

# Appliquer le masquage
masking_layer = Masking(mask_value=unknown_value)(reshaped_input)

# Ajout des couches LSTM
lstm_layer = LSTM(64, return_sequences=True)(masking_layer)
lstm_layer = BatchNormalization()(lstm_layer)
lstm_layer = Dropout(0.5)(lstm_layer)
lstm_layer = LSTM(64, return_sequences=False)(lstm_layer)
lstm_layer = BatchNormalization()(lstm_layer)
lstm_layer = Dropout(0.5)(lstm_layer)

# Ajouter une couche dense pour les prédictions
output = Dense(num_classes, activation='softmax')(lstm_layer)

# Créer le modèle
model = Model(inputs=input_seq, outputs=output)

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2564)]            0         
                                                                 
 reshape_1 (Reshape)         (None, 2564, 1)           0         
                                                                 
 masking_1 (Masking)         (None, 2564, 1)           0         
                                                                 
 lstm_2 (LSTM)               (None, 2564, 64)          16896     
                                                                 
 batch_normalization_2 (Bat  (None, 2564, 64)          256       
 chNormalization)                                                
                                                                 
 dropout_2 (Dropout)         (None, 2564, 64)          0         
                                                           

In [35]:
from tensorflow.keras.optimizers import Adam

# Utiliser un optimiseur avec un taux d'apprentissage ajusté
optimizer = Adam(learning_rate=0.001)

# Compiler le modèle avec l'optimiseur ajusté
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [36]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Calculer les poids de classe
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Convertir les poids de classe en un dictionnaire
class_weight_dict = dict(enumerate(class_weights))


In [37]:
print(len(X_train))
print(len(y_train))

2236
2236


In [38]:
import tensorflow as tf

# Print TensorFlow version
print("TensorFlow Version:", tf.__version__)

# Check if GPU is available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("Num GPUs Available: ", len(gpus))
    print("TensorFlow is using the GPU")
else:
    print("Num GPUs Available: ", 0)
    print("TensorFlow is not using the GPU")


#Je n'ai pas les bonnes versions de cuda et cudnn, elles ne sont pas compatible avec tensor flow j'ai donc laissée tomber l'idée d'utiliser le gpu 


TensorFlow Version: 2.14.0
Num GPUs Available:  0
TensorFlow is not using the GPU


In [39]:
# Entraîner le modèle 
history = model.fit(
    # X_train,
    # y_train,
    X_train_padded, 
    y_train_one_hot, 
    epochs=100, 
    batch_size=32, 
    validation_split=0.2, 
    # class_weight=class_weight_dict
)


Epoch 1/100
56/56 [==============================] - 225s 4s/step - loss: 5.5260 - accuracy: 0.0095 - val_loss: 5.2154 - val_accuracy: 0.0022
Epoch 2/100
56/56 [==============================] - 232s 4s/step - loss: 5.2398 - accuracy: 0.0123 - val_loss: 5.1767 - val_accuracy: 0.0045
Epoch 3/100
56/56 [==============================] - 221s 4s/step - loss: 5.0754 - accuracy: 0.0280 - val_loss: 5.1447 - val_accuracy: 0.0112
Epoch 4/100
56/56 [==============================] - 233s 4s/step - loss: 4.8775 - accuracy: 0.0391 - val_loss: 5.0835 - val_accuracy: 0.0045
Epoch 5/100
56/56 [==============================] - 236s 4s/step - loss: 4.7398 - accuracy: 0.0403 - val_loss: 4.9796 - val_accuracy: 0.0156
Epoch 6/100
56/56 [==============================] - 232s 4s/step - loss: 4.6018 - accuracy: 0.0498 - val_loss: 4.9030 - val_accuracy: 0.0268
Epoch 7/100
56/56 [==============================] - 239s 4s/step - loss: 4.4270 - accuracy: 0.0660 - val_loss: 5.1129 - val_accuracy: 0.0268
Epoch 

In [41]:
output_path = r'C:\Users\Jean\Documents\EIGSI\4a\Python for datascience\PROJET STARCRAFT\submission.csv'

def make_predictions_and_save(model, X_test, label_encoder, output_path):
    y_pred = model.predict(X_test_padded)
    y_pred_labels = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))
    
    submission_df = pd.DataFrame({
        'RowId': range(len(y_pred_labels)),  
        'Prediction': y_pred_labels
    })
    
    submission_df.to_csv(output_path, index=False)
    print(f"Submission file saved to {output_path}")

make_predictions_and_save(model, X_test_padded, name_encoder, output_path)

9/9 [==============================] - 13s 1s/step
Submission file saved to C:\Users\Jean\Documents\EIGSI\4a\Python for datascience\PROJET STARCRAFT\submission.csv


In [44]:
print(len(y_pred))

264


In [46]:
# Prédire les noms pour le jeu de test
y_pred = model.predict(X_test_padded)

# Convertir les prédictions en étiquettes de noms
y_pred_labels = name_encoder.inverse_transform(np.argmax(y_pred, axis=1))

# Afficher les prédictions
print(len(y_pred_labels))

9/9 [==============================] - 8s 852ms/step
264


In [47]:
print(len(y_pred))

264
